In [1]:
import sys
import os

sys.path.append(os.path.join(
    r'C:\Users\AA2-PC2\Software\NI-experiment-control - Atom Array\nistreamer'
))

In [2]:
from nistreamer.streamer import NIStreamer
from nistreamer.utils import iplot, RendOption
import numpy as np

In [3]:
ni_streamer = NIStreamer()

In [4]:
do_card = ni_streamer.add_do_card(max_name='PXI1Slot6', samp_rate=10e6)

do_bank = []
for port_idx in range(4):
    for line_idx in range(8):
        do_bank.append(do_card.add_chan_(
            port_idx=port_idx,
            line_idx=line_idx
        ))

In [5]:
len(do_bank)

32

### Back-to-back instructions

In [6]:
ni_streamer.clear_edit_cache()

total_dur = 10.0
instr_dur = 10e-3
instr_num = int(total_dur // instr_dur)

t = 0
for idx in range(instr_num):
    
    print(f'Adding instruction {idx} ===============')
        
    for do_chan in do_bank:
        do_chan.high(t=t, dur=instr_dur)
        # do_chan.go_high(t=t)
        # do_chan.go_low(t=t + instr_dur - buf)
        
    t += instr_dur

Adding instruction 0 ===============
Adding instruction 1 ===============
Adding instruction 2 ===============
Adding instruction 3 ===============
Adding instruction 4 ===============
Adding instruction 5 ===============
Adding instruction 6 ===============
Adding instruction 7 ===============
Adding instruction 8 ===============
Adding instruction 9 ===============
Adding instruction 10 ===============


PanicException: Did not encounter 1-tick collision on the left

Previous instruction on the left: 
	 t = 0.09 s
	 dur = 0.01 s
	 start_pos = 900000 
	 end_pos = 1000000

New instruction:
	 t = 0.09999999999999999 s
	 dur = 0.01 s
	 start_pos = 1000000 
	 end_pos = 1100000 

### Test notes

#### 1. No rounding (truncation), `end_pos = (dur as usize) + start_pos` - panics

```Rust
fn add_instr(&mut self, func: Instruction, t: f64, dur_spec: Option<(f64, bool)>) {
        // Convert floating-point start and end times to sample clock ticks
        let start_pos = (t * self.samp_rate()) as usize;
        let end_spec = match dur_spec {
            Some((dur, keep_val)) => {
                let end_pos = ((dur) * self.samp_rate()) as usize + start_pos;
                // Sanity check - pulse length is at leas 1 clock period or longer
                if end_pos - start_pos < 1 {
                    // panic message
                };
                Some((end_pos, keep_val))
            },
            None => None,
        };
```

```
Adding instruction 0 ===============
Adding instruction 1 ===============
Adding instruction 2 ===============
Adding instruction 3 ===============
Adding instruction 4 ===============
Adding instruction 5 ===============
Adding instruction 6 ===============
Adding instruction 7 ===============
Adding instruction 8 ===============
Adding instruction 9 ===============
Adding instruction 10 ===============

---------------------------------------------------------------------------
PanicException                            Traceback (most recent call last)
<ipython-input-6-c4ca24defb39> in <module>
     11 
     12     for do_chan in do_bank:
---> 13         do_chan.high(t=t, dur=instr_dur)
     14         # do_chan.go_high(t=t)
     15         # do_chan.go_low(t=t + instr_dur - buf)

~\Software\NI-experiment-control - Atom Array\nistreamer\nistreamer\channel.py in high(self, t, dur)
    183 
    184     def high(self, t, dur):
--> 185         self._dll.high(
    186             dev_name=self._card_max_name,
    187             chan_name=self.chan_name,

PanicException: Encountered 1-tick collision on the left

Previous instruction on the left: 
	 t = 0.09 s
	 dur = 0.01 s
	 start_pos = 900000 
	 end_pos = 1000000

New instruction:
	 t = 0.09999999999999999 s
	 dur = 0.01 s
	 start_pos = 999999 
	 end_pos = 1099999 
```

#### 2. Rounding, `end_pos = (dur as usize) + start_pos` - no panic

```Rust
fn add_instr(&mut self, func: Instruction, t: f64, dur_spec: Option<(f64, bool)>) {
        // Convert floating-point start and end times to sample clock ticks
        let start_pos = (t * self.samp_rate()).round() as usize;
        let end_spec = match dur_spec {
            Some((dur, keep_val)) => {
                let end_pos = ((dur) * self.samp_rate()).round() as usize + start_pos;
                // Sanity check - pulse length is at leas 1 clock period or longer
                if end_pos - start_pos < 1 {
                    // panic message
                };
                Some((end_pos, keep_val))
            },
            None => None,
        };
```

```
Adding instruction 0 ===============
Adding instruction 1 ===============
Adding instruction 2 ===============
Adding instruction 3 ===============
Adding instruction 4 ===============
Adding instruction 5 ===============
Adding instruction 6 ===============
Adding instruction 7 ===============
Adding instruction 8 ===============
Adding instruction 9 ===============
Adding instruction 10 ===============

---------------------------------------------------------------------------
PanicException                            Traceback (most recent call last)
<ipython-input-6-c4ca24defb39> in <module>
     11 
     12     for do_chan in do_bank:
---> 13         do_chan.high(t=t, dur=instr_dur)
     14         # do_chan.go_high(t=t)
     15         # do_chan.go_low(t=t + instr_dur - buf)

~\Software\NI-experiment-control - Atom Array\nistreamer\nistreamer\channel.py in high(self, t, dur)
    183 
    184     def high(self, t, dur):
--> 185         self._dll.high(
    186             dev_name=self._card_max_name,
    187             chan_name=self.chan_name,

PanicException: Did not encounter 1-tick collision on the left

Previous instruction on the left: 
	 t = 0.09 s
	 dur = 0.01 s
	 start_pos = 900000 
	 end_pos = 1000000

New instruction:
	 t = 0.09999999999999999 s
	 dur = 0.01 s
	 start_pos = 1000000 
	 end_pos = 1100000 


```

#### 3. No rounding (truncation), `end_pos = (t + dur) as usize` - no panic

```Rust
fn add_instr(&mut self, func: Instruction, t: f64, dur_spec: Option<(f64, bool)>) {
        // Convert floating-point start and end times to sample clock ticks
        let start_pos = (t * self.samp_rate()) as usize;
        let end_spec = match dur_spec {
            Some((dur, keep_val)) => {
                let end_pos = ((dur + t) * self.samp_rate()) as usize;
                // Sanity check - pulse length is at leas 1 clock period or longer
                if end_pos - start_pos < 1 {// panic message};
                Some((end_pos, keep_val))
            },
            None => None,
        };
```

```
Adding instruction 0 ===============
Adding instruction 1 ===============
Adding instruction 2 ===============
Adding instruction 3 ===============
Adding instruction 4 ===============
Adding instruction 5 ===============
Adding instruction 6 ===============
Adding instruction 7 ===============
Adding instruction 8 ===============
Adding instruction 9 ===============
Adding instruction 10 ===============

---------------------------------------------------------------------------
PanicException                            Traceback (most recent call last)
<ipython-input-6-c4ca24defb39> in <module>
     11 
     12     for do_chan in do_bank:
---> 13         do_chan.high(t=t, dur=instr_dur)
     14         # do_chan.go_high(t=t)
     15         # do_chan.go_low(t=t + instr_dur - buf)

~\Software\NI-experiment-control - Atom Array\nistreamer\nistreamer\channel.py in high(self, t, dur)
    183 
    184     def high(self, t, dur):
--> 185         self._dll.high(
    186             dev_name=self._card_max_name,
    187             chan_name=self.chan_name,

PanicException: Did not encounter 1-tick collision on the left

Previous instruction on the left: 
	 t = 0.09 s
	 dur = 0.01 s
	 start_pos = 900000 
	 end_pos = 999999

New instruction:
	 t = 0.09999999999999999 s
	 dur = 0.01 s
	 start_pos = 999999 
	 end_pos = 1099999 


```

#### 4. Rounding, `end_pos = (t + dur) as usize` - no panic

```Rust
fn add_instr(&mut self, func: Instruction, t: f64, dur_spec: Option<(f64, bool)>) {
        // Convert floating-point start and end times to sample clock ticks
        let start_pos = (t * self.samp_rate()).round() as usize;
        let end_spec = match dur_spec {
            Some((dur, keep_val)) => {
                let end_pos = ((t + dur) * self.samp_rate()).round() as usize;
                // Sanity check - pulse length is at leas 1 clock period or longer
                if end_pos - start_pos < 1 {// panic message}
                Some((end_pos, keep_val))
            },
            None => None,
        };
        let mut new_instr_book = InstrBook::new(start_pos, end_spec, func);
```

```
Adding instruction 0 ===============
Adding instruction 1 ===============
Adding instruction 2 ===============
Adding instruction 3 ===============
Adding instruction 4 ===============
Adding instruction 5 ===============
Adding instruction 6 ===============
Adding instruction 7 ===============
Adding instruction 8 ===============
Adding instruction 9 ===============
Adding instruction 10 ===============

---------------------------------------------------------------------------
PanicException                            Traceback (most recent call last)
<ipython-input-6-c4ca24defb39> in <module>
     11 
     12     for do_chan in do_bank:
---> 13         do_chan.high(t=t, dur=instr_dur)
     14         # do_chan.go_high(t=t)
     15         # do_chan.go_low(t=t + instr_dur - buf)

~\Software\NI-experiment-control - Atom Array\nistreamer\nistreamer\channel.py in high(self, t, dur)
    183 
    184     def high(self, t, dur):
--> 185         self._dll.high(
    186             dev_name=self._card_max_name,
    187             chan_name=self.chan_name,

PanicException: Did not encounter 1-tick collision on the left

Previous instruction on the left: 
	 t = 0.09 s
	 dur = 0.01 s
	 start_pos = 900000 
	 end_pos = 1000000

New instruction:
	 t = 0.09999999999999999 s
	 dur = 0.01 s
	 start_pos = 1000000 
	 end_pos = 1100000 
```

### Summary

Test confirmed that truncation can indeed lead to an unexpected jump by 1 down for a float "nominally equal" to an integer.

Switching to rounding moves the instability values to half-integers, where jump would be +-1/2.

Once selected between truncation or rounding, `start_pos` is calculated in only one way (shown for rounding):

```
let start_pos = (t * self.samp_rate()).round() as usize;
```

But there are still two possible ways of calculating `end_pos`:
- by rounding `dur` first and then adding to `start_pos`  
 `let end_pos = (dur * self.samp_rate()).round() as usize + start_pos`
 

- by summing `t + dur` as floats first and then converting to integer:  
 `let end_pos = ((t + dur) * self.samp_rate()) as usize`

**The second way is preferred.**

- it makes behavior of the second edge more intuitive. 
If (t+dur) was close to a clock tick, it will reliably be rounded there. On the contrary, first way would lead to unpredictable/unintuitive result for the second edge. For example, `t_start=1.5*clock_period` and `dur=1.5*clock_period` intuitively should reliably give second edge at 3 since it precisely matches this tick, but rounding + summing will give it at 4 (both t_start and dur will be rounded to 2).


- the first way increases risks of virtual 1-tick collisions for back-to-back pulses. Because it makes `end_pos` of the previous pulse and `start_pos` of the next pulse be calculated through different "float->int conversion path". For `end_pos`, start time and duration are converted independently and then added, while for `start_pos` of the next pulse (start time + dur) are added as floats to give `t` first and then this `t` is converted. Using 2nd way makes both `end_pos` and `start_pos` be calculaded the same way, so if jump occurs, it is likely to be identical on both and not lead to a collision.

This is precisely why the case `3. No rounding (truncation), end_pos = (t + dur) as usize - no panic` does not panic and case `1. No rounding (truncation), `end_pos = (dur as usize) + start_pos` - panics` does. Jump does happen in both cases, but for test "3" both `end_pos` and `start_pos` are shifted together.


```
// [x] denotes truncation of x

// 1st way - happens in test (1) - panic
end_pos = [90 ms] + [10 ms] = 90,000,0 + 10,000,0 = 100,000,0
start_pos = [90 ms + 10 ms] = [99.(9) ms] = 99,999,9

// 2nd way - happens in test (3) - no panic
end_pos = [90 ms + 10 ms] = [99.(9) ms] = 99,999,9
start_pos = [90 ms + 10 ms] = [99.(9) ms] = 99,999,9
```

Note that such "synchronization" of jumps may be contingent on start time `t` for the second pulse be calculated precisely as `start t of the first + duration of the first` since that fully reproduces arithmetic sequence for the first pulse end time `t + dur`. If start of the second pulse is obtained by another sequence, precision errors there might still break "synchronization".

Overall, using:
- rounding to move instability to half-integers
- rounding (t + dur) to make "conversion path" similar for both `end_pos` and `start_pos`

moves the unstable regions to a more intuitive place and reduces chances of matching `end_pos` and `start_pos` to jump across each other.

Still, jumps could in principle occur (precisely half-integers, different arithmetics path for 2nd pulse start time) and it should be handled.